<div style="background-color:#B4DBE9; color:#636363;">
    <h1><center>Introducción</center></h1>
</div>

** El agua segura y fácilmente disponible ** es importante para la salud pública, ya sea que se use para beber, uso doméstico, producción de alimentos o fines recreativos.El suministro y el saneamiento de agua mejorado, y una mejor gestión de los recursos hídricos, pueden * impulsar el crecimiento económico de los países * y puede contribuir en gran medida a la reducción de la pobreza.

El agua contaminada y el saneamiento deficiente están vinculados a la transmisión de enfermedades como ** cólera, diarrea, disentería, hepatitis A, tifoidea y polio **.Los servicios de agua y saneamiento ausentes, inadecuados o administrados de manera inadecuada exponen a las personas a los riesgos de salud prevenibles.Este es particularmente el caso en los centros de atención médica, donde tanto los pacientes como el personal se ponen en riesgo adicional de infección y enfermedad cuando faltan los servicios de agua, saneamiento e higiene.A nivel mundial, ** 15%** de pacientes desarrollan una infección durante una estadía en el hospital, con la proporción mucho mayor en los países de bajos ingresos.

Por lo tanto, me inspiré en esto para usar este conjunto de datos ** de calidad del agua ** para comprender qué considera al agua segura y potable y aplicarle el aprendizaje automático para distinguir entre agua potable y no potable.

<div style="background-color:#B4DBE9;">
    <center><img src="https://raw.githubusercontent.com/jaykumar1607/Water-Quality/main/water_sanitation.gif">
</div>

---

<div style="background-color:#B4DBE9; color:#636363;">
    <h1><center>Descripción de la característica</center></h1>
</div>

1. **ph**: pH de 1. agua (0 a 14).

2. **Dureza**: Capacidad del agua para precipitar el jabón en Mg/L.

3. **Sólidos**: Sólidos disueltos totales en PPM.

4. **Cloraminas**: Cantidad de cloraminas en PPM.

5. **Sulfato**: Cantidad de sulfatos disueltos en Mg/L.

6. **Conductividad**: conductividad eléctrica del agua en μs/cm.

7. **Organic_carbon**: cantidad de carbono orgánico en PPM.

8. **Trihalometanos**: cantidad de trihalometanos en μg/L.

9. **Turbidez**: Medida de propiedad emitida de luz del agua en NTU.

10. **Potabilidad**: Indica si el agua es segura para el consumo humano.Potable - 1 y no potable - 0

---

<div style="background-color:#B4DBE9; color:#636363;">
    <h1><center>Librerías</center></h1>
</div>

In [ ]:
# Basic Libraries
import numpy as np
import pandas as pd
from warnings import filterwarnings
from collections import Counter

# Visualizations Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.offline as pyo
import plotly.express as px
import plotly.graph_objs as go
pyo.init_notebook_mode()
import plotly.figure_factory as ff
import missingno as msno

# Data Pre-processing Libraries
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split

# Modelling Libraries
from sklearn.linear_model import LogisticRegression,RidgeClassifier,SGDClassifier,PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC,LinearSVC,NuSVC
from sklearn.neighbors import KNeighborsClassifier,NearestCentroid
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.ensemble import VotingClassifier

# Evaluation & CV Libraries
from sklearn.metrics import precision_score,accuracy_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV,RepeatedStratifiedKFold

---

<div style="background-color:#B4DBE9; color:#636363;">
    <h1><center>Colores</center></h1>
</div>

In [ ]:
colors_blue = ["#132C33", "#264D58", '#17869E', '#51C4D3', '#B4DBE9']
colors_dark = ["#1F1F1F", "#313131", '#636363', '#AEAEAE', '#DADADA']
colors_green = ['#01411C','#4B6F44','#4F7942','#74C365','#D0F0C0']
sns.palplot(colors_blue)
sns.palplot(colors_green)
sns.palplot(colors_dark)

---

<div style="background-color:#B4DBE9; color:#636363;">
    <h1><center>Importando el set de datos</center></h1>
</div>

In [ ]:
df=pd.read_csv('../input/water-potability/water_potability.csv')

In [ ]:
df.info()

In [ ]:
df.head()

---

<div style="background-color:#B4DBE9; color:#636363;">
    <h1><center>Visualizaciones</center></h1>
</div>

In [ ]:
d= pd.DataFrame(df['Potability'].value_counts())
fig = px.pie(d,values='Potability',names=['Not Potable','Potable'],hole=0.4,opacity=0.6,
            color_discrete_sequence=[colors_green[3],colors_blue[3]],
             labels={'label':'Potability','Potability':'No. Of Samples'})

fig.add_annotation(text='We can resample the data<br> to get a balanced dataset',
                   x=1.2,y=0.9,showarrow=False,font_size=12,opacity=0.7,font_family='monospace')
fig.add_annotation(text='Potability',
                   x=0.5,y=0.5,showarrow=False,font_size=14,opacity=0.7,font_family='monospace')

fig.update_layout(
    font_family='monospace',
    title=dict(text='Q. How many samples of water are Potable?',x=0.47,y=0.98,
               font=dict(color=colors_dark[2],size=20)),
    legend=dict(x=0.37,y=-0.05,orientation='h',traceorder='reversed'),
    hoverlabel=dict(bgcolor='white'))

fig.update_traces(textposition='outside', textinfo='percent+label')

fig.show()

Veamos la distribución de las características.

**Dureza de agua**: La definición simple de dureza del agua es la cantidad de calcio y magnesio disuelto en el agua.El agua dura es alta en minerales disueltos, en gran medida de calcio y magnesio.Es posible que haya sentido los efectos del agua dura, literalmente, la última vez que se lavó las manos.Dependiendo de la dureza de su agua, después de usar jabón para lavar, es posible que haya sentido que quedaba una película de residuos en sus manos.En agua dura, el jabón reacciona con el calcio (que es relativamente alto en agua dura) para formar "escoria de jabón".Cuando se usa agua dura, se necesita más jabón o detergente para limpiar las cosas, ya sea sus manos, cabello o su lavandería.

In [ ]:
fig = px.histogram(df,x='Hardness',y=Counter(df['Hardness']),color='Potability',template='plotly_white',
                  marginal='box',opacity=0.7,nbins=100,color_discrete_sequence=[colors_green[3],colors_blue[3]],
                  barmode='group',histfunc='count')

fig.add_vline(x=151, line_width=1, line_color=colors_dark[1],line_dash='dot',opacity=0.7)
fig.add_vline(x=301, line_width=1, line_color=colors_dark[1],line_dash='dot',opacity=0.7)
fig.add_vline(x=76, line_width=1, line_color=colors_dark[1],line_dash='dot',opacity=0.7)

fig.add_annotation(text='<76 mg/L is<br> considered soft',x=40,y=130,showarrow=False,font_size=9)
fig.add_annotation(text='Between 76 and 150<br> (mg/L) is<br>moderately hard',x=113,y=130,showarrow=False,font_size=9)
fig.add_annotation(text='Between 151 and 300 (mg/L)<br> is considered hard',x=250,y=130,showarrow=False,font_size=9)
fig.add_annotation(text='>300 mg/L is<br> considered very hard',x=340,y=130,showarrow=False,font_size=9)

fig.update_layout(
    font_family='monospace',
    title=dict(text='Hardness Distribution',x=0.53,y=0.95,
               font=dict(color=colors_dark[2],size=20)),
    xaxis_title_text='Hardness (mg/L)',
    yaxis_title_text='Count',
    legend=dict(x=1,y=0.96,bordercolor=colors_dark[4],borderwidth=0,tracegroupgap=5),
    bargap=0.3,
)
fig.show()

**pH level:** El pH del agua es una medida del equilibrio ácido -base y, en la mayoría de las aguas naturales, está controlado por el sistema de equilibrio de dióxido de carbono -bicarbonato -carbonato.Por lo tanto, una mayor concentración de dióxido de carbono reducirá el pH, mientras que una disminución hará que aumente.La temperatura también afectará los equilibrios y el pH.En agua pura, se produce una disminución en el pH de aproximadamente 0,45 ya que la temperatura se eleva en 25 ° C.En el agua con una capacidad de amortiguación impartida por bicarbonato, carbonato e iones hidroxilo, este efecto de temperatura se modifica (Apha, 1989).El pH de la mayoría de las aguas potables se encuentra dentro del rango 6.5–8.5.Las aguas naturales pueden ser de pH más bajo, como resultado de, por ejemplo, lluvia ácida o pH más alto en áreas de piedra caliza.

In [ ]:
fig = px.histogram(df,x='ph',y=Counter(df['ph']),color='Potability',template='plotly_white',
                  marginal='box',opacity=0.7,nbins=100,color_discrete_sequence=[colors_green[3],colors_blue[3]],
                  barmode='group',histfunc='count')

fig.add_vline(x=7, line_width=1, line_color=colors_dark[1],line_dash='dot',opacity=0.7)

fig.add_annotation(text='<7 is Acidic',x=4,y=70,showarrow=False,font_size=10)
fig.add_annotation(text='>7 is Basic',x=10,y=70,showarrow=False,font_size=10)


fig.update_layout(
    font_family='monospace',
    title=dict(text='pH Level Distribution',x=0.5,y=0.95,
               font=dict(color=colors_dark[2],size=20)),
    xaxis_title_text='pH Level',
    yaxis_title_text='Count',
    legend=dict(x=1,y=0.96,bordercolor=colors_dark[4],borderwidth=0,tracegroupgap=5),
    bargap=0.3,
)
fig.show()

**TDS**: TDS significa concentración de partículas disueltas o sólidos en el agua.TDS comprende sales inorgánicas como calcio, magnesio, cloruros, sulfatos, bicarbonatos, etc., junto con muchos más compuestos inorgánicos que se disuelven fácilmente en el agua.

In [ ]:
fig = px.histogram(df,x='Solids',y=Counter(df['Solids']),color='Potability',template='plotly_white',
                  marginal='box',opacity=0.7,nbins=100,color_discrete_sequence=[colors_green[3],colors_blue[3]],
                  barmode='group',histfunc='count')

fig.update_layout(
    font_family='monospace',
    title=dict(text='Distribution Of Total Dissolved Solids',x=0.5,y=0.95,
               font=dict(color=colors_dark[2],size=20)),
    xaxis_title_text='Dissolved Solids (ppm)',
    yaxis_title_text='Count',
    legend=dict(x=1,y=0.96,bordercolor=colors_dark[4],borderwidth=0,tracegroupgap=5),
    bargap=0.3,
)
fig.show()

**Chloramines**: Las cloraminas (también conocidas como desinfección secundaria) son desinfectantes utilizados para tratar el agua potable y ellos:

* Se forman más comúnmente cuando se agrega amoníaco al cloro para tratar el agua potable.
* Proporcione una desinfección más duradera a medida que el agua se mueve a través de las tuberías a los consumidores.

Las cloraminas han sido utilizadas por los servicios de agua desde la década de 1930.

In [ ]:
fig = px.histogram(df,x='Chloramines',y=Counter(df['Chloramines']),color='Potability',template='plotly_white',
                  marginal='box',opacity=0.7,nbins=100,color_discrete_sequence=[colors_green[3],colors_blue[3]],
                  barmode='group',histfunc='count')

fig.add_vline(x=4, line_width=1, line_color=colors_dark[1],line_dash='dot',opacity=0.7)

fig.add_annotation(text='<4 ppm is considered<br> safe for drinking',x=1.8,y=90,showarrow=False)

fig.update_layout(
    font_family='monospace',
    title=dict(text='Chloramines Distribution',x=0.53,y=0.95,
               font=dict(color=colors_dark[2],size=20)),
    xaxis_title_text='Chloramines (ppm)',
    yaxis_title_text='Count',
    legend=dict(x=1,y=0.96,bordercolor=colors_dark[4],borderwidth=0,tracegroupgap=5),
    bargap=0.3,
)
fig.show()

**Sulfate**: El sulfato (SO4) se puede encontrar en casi toda el agua natural.El origen de la mayoría de los compuestos de sulfato es la oxidación de los minerales de sulfito, la presencia de lutitas o los desechos industriales.
El sulfato es uno de los principales componentes disueltos de la lluvia.Las altas concentraciones de sulfato en el agua que bebemos pueden tener un efecto laxante cuando se combinan con calcio y magnesio, los dos componentes más comunes de la dureza.

In [ ]:
fig = px.histogram(df,x='Sulfate',y=Counter(df['Sulfate']),color='Potability',template='plotly_white',
                  marginal='box',opacity=0.7,nbins=100,color_discrete_sequence=[colors_green[3],colors_blue[3]],
                  barmode='group',histfunc='count')

fig.add_vline(x=250, line_width=1, line_color=colors_dark[1],line_dash='dot',opacity=0.7)

fig.add_annotation(text='<250 mg/L is considered<br> safe for drinking',x=175,y=90,showarrow=False)

fig.update_layout(
    font_family='monospace',
    title=dict(text='Sulfate Distribution',x=0.53,y=0.95,
               font=dict(color=colors_dark[2],size=20)),
    xaxis_title_text='Sulfate (mg/L)',
    yaxis_title_text='Count',
    legend=dict(x=1,y=0.96,bordercolor=colors_dark[4],borderwidth=0,tracegroupgap=5),
    bargap=0.3,
)
fig.show()

**Conductivity**: La conductividad es una medida de la capacidad del agua para pasar una corriente eléctrica.Debido a que las sales disueltas y otros productos químicos inorgánicos conducen corriente eléctrica, la conductividad aumenta a medida que aumenta la salinidad.Los compuestos orgánicos como el aceite no conducen muy bien la corriente eléctrica y, por lo tanto, tienen una baja conductividad cuando están en agua.La conductividad también se ve afectada por la temperatura: cuanto más cálido sea el agua, mayor es la conductividad.

In [ ]:
fig = px.histogram(df,x='Conductivity',y=Counter(df['Conductivity']),color='Potability',template='plotly_white',
                  marginal='box',opacity=0.7,nbins=100,color_discrete_sequence=[colors_green[3],colors_blue[3]],
                  barmode='group',histfunc='count')

fig.add_annotation(text='The Conductivity range <br> is safe for both (200-800),<br> Potable and Non-Potable water',
                   x=600,y=90,showarrow=False)

fig.update_layout(
    font_family='monospace',
    title=dict(text='Conductivity Distribution',x=0.5,y=0.95,
               font=dict(color=colors_dark[2],size=20)),
    xaxis_title_text='Conductivity (μS/cm)',
    yaxis_title_text='Count',
    legend=dict(x=1,y=0.96,bordercolor=colors_dark[4],borderwidth=0,tracegroupgap=5),
    bargap=0.3,
)
fig.show()

**Organic Carbon**: Los contaminantes orgánicos (sustancias orgánicas naturales, insecticidas, herbicidas y otros productos químicos agrícolas) ingresan a las vías fluviales en la escorrentía de lluvia.Las aguas residuales nacionales e industriales también contribuyen con contaminantes orgánicos en varias cantidades.Como resultado de derrames o fugas accidentales, los desechos orgánicos industriales pueden ingresar a las corrientes.Algunos de los contaminantes pueden no ser eliminados por completo por procesos de tratamiento;Por lo tanto, podrían convertirse en un problema para beber fuentes de agua.Es importante conocer el contenido orgánico en una vía fluvial.

In [ ]:
fig = px.histogram(df,x='Organic_carbon',y=Counter(df['Organic_carbon']),color='Potability',template='plotly_white',
                  marginal='box',opacity=0.7,nbins=100,color_discrete_sequence=[colors_green[3],colors_blue[3]],
                  barmode='group',histfunc='count')

fig.add_vline(x=10, line_width=1, line_color=colors_dark[1],line_dash='dot',opacity=0.7)

fig.add_annotation(text='Typical Organic Carbon<br> level is upto 10 ppm',x=5.3,y=110,showarrow=False)

fig.update_layout(
    font_family='monospace',
    title=dict(text='Organic Carbon Distribution',x=0.5,y=0.95,
               font=dict(color=colors_dark[2],size=20)),
    xaxis_title_text='Organic Carbon (ppm)',
    yaxis_title_text='Count',
    legend=dict(x=1,y=0.96,bordercolor=colors_dark[4],borderwidth=0,tracegroupgap=5),
    bargap=0.3,
)
fig.show()

**Trihalomethanes**: Los trihalometanos (THM) son el resultado de una reacción entre el cloro utilizado para desinfectar el agua del grifo y la materia orgánica natural en el agua.A niveles elevados, los THM se han asociado con efectos negativos para la salud, como el cáncer y los resultados reproductivos adversos.

In [ ]:
fig = px.histogram(df,x='Trihalomethanes',y=Counter(df['Trihalomethanes']),color='Potability',template='plotly_white',
                  marginal='box',opacity=0.7,nbins=100,color_discrete_sequence=[colors_green[3],colors_blue[3]],
                  barmode='group',histfunc='count')

fig.add_vline(x=80, line_width=1, line_color=colors_dark[1],line_dash='dot',opacity=0.7)

fig.add_annotation(text='Upper limit of Trihalomethanes<br> level is 80 μg/L',x=115,y=90,showarrow=False)

fig.update_layout(
    font_family='monospace',
    title=dict(text='Trihalomethanes Distribution',x=0.5,y=0.95,
               font=dict(color=colors_dark[2],size=20)),
    xaxis_title_text='Trihalomethanes (μg/L)',
    yaxis_title_text='Count',
    legend=dict(x=1,y=0.96,bordercolor=colors_dark[4],borderwidth=0,tracegroupgap=5),
    bargap=0.3,
)
fig.show()

**Turbidity**: La turbidez es la medida de la claridad relativa de un líquido.Es una característica óptica del agua y es una medida de la cantidad de luz dispersa por material en el agua cuando una luz se brilla a través de la muestra de agua.Cuanto mayor sea la intensidad de la luz dispersa, mayor es la turbidez.El material que hace que el agua sea turbia incluya arcilla, limo, materia inorgánica y orgánica muy pequeña, algas, compuestos orgánicos de color disuelto y plancton y otros organismos microscópicos.

In [ ]:
fig = px.histogram(df,x='Turbidity',y=Counter(df['Turbidity']),color='Potability',template='plotly_white',
                  marginal='box',opacity=0.7,nbins=100,color_discrete_sequence=[colors_green[3],colors_blue[3]],
                  barmode='group',histfunc='count')

fig.add_vline(x=5, line_width=1, line_color=colors_dark[1],line_dash='dot',opacity=0.7)

fig.add_annotation(text='<5 NTU Turbidity is<br> considered safe',x=6,y=90,showarrow=False)

fig.update_layout(
    font_family='monospace',
    title=dict(text='Turbidity Distribution',x=0.5,y=0.95,
               font=dict(color=colors_dark[2],size=20)),
    xaxis_title_text='Turbidity (NTU)',
    yaxis_title_text='Count',
    legend=dict(x=1,y=0.96,bordercolor=colors_dark[4],borderwidth=0,tracegroupgap=5),
    bargap=0.3,
)
fig.show()

La matriz de gráficos de dispersión ayuda a descubrir la correlación entre todas las características.

In [ ]:
fig = px.scatter_matrix(df,df.drop('Potability',axis=1),height=1250,width=1250,template='plotly_white',opacity=0.7,
                        color_discrete_sequence=[colors_blue[3],colors_green[3]],color='Potability',
                       symbol='Potability',color_continuous_scale=[colors_green[3],colors_blue[3]])

fig.update_layout(font_family='monospace',font_size=10,
                  coloraxis_showscale=False,
                 legend=dict(x=0.02,y=1.07,bgcolor=colors_dark[4]),
                 title=dict(text='Scatter Plot Matrix b/w Features',x=0.5,y=0.97,
                   font=dict(color=colors_dark[2],size=24)))
fig.show()

Como podemos ver, parece haber muy menos correlación entre todas las características.

In [ ]:
cor=df.drop('Potability',axis=1).corr()
cor

Hagamos un mapa de calor para visualizar la correlación.

In [ ]:
fig = px.imshow(cor,height=800,width=800,color_continuous_scale=colors_blue,template='plotly_white')

fig.update_layout(font_family='monospace',
                title=dict(text='Correlation Heatmap',x=0.5,y=0.93,
                             font=dict(color=colors_dark[2],size=24)),
                coloraxis_colorbar=dict(len=0.85,x=1.1) 
                 )

fig.show()

---

<div style="background-color:#B4DBE9; color:#636363;">
    <h1><center>Preparación de los datos</center></h1>
</div>

<div style="background-color:#B4DBE9; color:#636363;">
    <h3><center>Lidiando con los valores faltantes</center></h3>
</div>

In [ ]:
fig = msno.matrix(df,color=(0,0.5,0.5))

Podemos ver que hay muchos valores faltantes en el conjunto de datos, así que intentaré lidiar con él.

In [ ]:
df.isnull().sum()

In [ ]:
df[df['Potability']==0].describe()

In [ ]:
df[df['Potability']==1].describe()

In [ ]:
df[df['Potability']==0][['ph','Sulfate','Trihalomethanes']].median()

In [ ]:
df[df['Potability']==1][['ph','Sulfate','Trihalomethanes']].median()

Podemos ver que la diferencia entre los valores medios y medios de agua potable y no potable es mínima.
Entonces usamos la mediana general de la característica para imputar los valores

In [ ]:
df['ph'].fillna(value=df['ph'].median(),inplace=True)
df['Sulfate'].fillna(value=df['Sulfate'].median(),inplace=True)
df['Trihalomethanes'].fillna(value=df['Trihalomethanes'].median(),inplace=True)

In [ ]:
df.isnull().sum()

<div style="background-color:#B4DBE9; color:#636363;">
    <h3><center>Estandarización de los datos</center></h3>
</div>

In [ ]:
X = df.drop('Potability',axis=1).values
y = df['Potability'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

<div style="background-color:#B4DBE9; color:#636363;">
    <h1><center>Modelling</center></h1>
</div>

<div style="background-color:#B4DBE9; color:#636363;">
    <h3><center>Spot checking</center></h3>
</div>

El spot checking es una estrategia valiosa en el análisis de datos porque permite a los analistas y científicos de datos identificar rápidamente los algoritmos más prometedores para un conjunto de datos específico. Al enfocarse en los modelos que inicialmente muestran un buen desempeño, se puede ahorrar tiempo y recursos durante la fase de desarrollo y optimización del modelo. La comprobación de Spot es un gran método para descubrir los modelos de referencia para nuestros datos.Es bastante fácil y toma muy menos tiempo.

In [ ]:
filterwarnings('ignore')
models =[("LR", LogisticRegression(max_iter=1000)),("SVC", SVC()),('KNN',KNeighborsClassifier(n_neighbors=10)),
         ("DTC", DecisionTreeClassifier()),("GNB", GaussianNB()),
        ("SGDC", SGDClassifier()),("Perc", Perceptron()),("NC",NearestCentroid()),
        ("Ridge", RidgeClassifier()),("NuSVC", NuSVC()),("BNB", BernoulliNB()),
         ('RF',RandomForestClassifier()),('ADA',AdaBoostClassifier()),
        ('XGB',GradientBoostingClassifier()),('PAC',PassiveAggressiveClassifier())]

results = []
names = []
finalResults = []

for name,model in models:
    model.fit(X_train, y_train)
    model_results = model.predict(X_test)
    score = precision_score(y_test, model_results,average='macro')
    results.append(score)
    names.append(name)
    finalResults.append((name,score))
    
finalResults.sort(key=lambda k:k[1],reverse=True)

In [ ]:
finalResults

Elegí los 5 principales modelos de referencia y realicé un ajuste de hiperparámetro.
2 modelos eclipsaron a otros modelos y eran bosques aleatorios y xgboost, por lo que los elijo para mi modelo final.

<div style="background-color:#B4DBE9; color:#636363;">
    <h3><center>Hyperparameter Tuning</center></h3>
</div>

El Hyperparameter Tuning (ajuste de hiperparámetros) es el proceso de buscar los valores óptimos para los hiperparámetros de un modelo de aprendizaje automático. Los hiperparámetros son parámetros que se configuran antes de entrenar el modelo y no se aprenden directamente del proceso de entrenamiento. Estos parámetros pueden tener un impacto significativo en el rendimiento del modelo.

Diferencia entre Parámetros y Hiperparámetros
Parámetros: Valores internos del modelo que se aprenden durante el entrenamiento, como los coeficientes en una regresión lineal o los pesos en una red neuronal.
Hiperparámetros: Configuraciones externas al modelo que deben establecerse antes del entrenamiento, como la tasa de aprendizaje, el número de árboles en un bosque aleatorio o la profundidad máxima de un árbol de decisión.


In [ ]:
model_params = {
    'XGB':
    {
        'model':GradientBoostingClassifier(),
        'params':
        {
            'learning_rate':[0.0001,0.001,0.01,0.1],
            'n_estimators':[100,200,500,1000],
            'max_features':['sqrt','log2'],
            'max_depth':list(range(11))
        }
    },
    'Random Forest':
    {
        'model':RandomForestClassifier(),
        'params':
        {
            'n_estimators':[10,50,100,200],
            'max_features':['auto','sqrt','log2'],
            'max_depth':list(range(1,11))
        }
    }
}

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=5,n_repeats=2)
scores=[]
for model_name,params in model_params.items():
    rs = RandomizedSearchCV(params['model'],params['params'],cv=cv,n_iter=20)
    rs.fit(X,y)
    scores.append([model_name,dict(rs.best_params_),rs.best_score_])
data=pd.DataFrame(scores,columns=['Model','Parameters','Score'])
data

Así que ahora tenemos los mejores parámetros para nuestro modelo de conjunto final.

---

<div style="background-color:#B4DBE9; color:#636363;">
    <h1><center>Modelo Final</center></h1>
</div>

¡He usado el VotingClassFier para conjunto de los modelos para obtener mejores resultados!

In [ ]:
param=data['Parameters']
model = VotingClassifier(estimators=[
                                     ('XGB',GradientBoostingClassifier(**param[0])),
                                     ('RF',RandomForestClassifier(**param[1])),
                                    ],voting='hard')

accuracy=[]
scaler = StandardScaler()
skf = RepeatedStratifiedKFold(n_splits=5,n_repeats=2)
skf.get_n_splits(X,y)

for train_index, test_index in skf.split(X,y):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    model.fit(X_train,y_train)
    predictions=model.predict(X_test)
    score=accuracy_score(y_test,predictions)
    accuracy.append(score)    

In [ ]:
np.mean(accuracy)

Creo que las características no están demostrando mucha ayuda para el modelo para distinguir entre las 2 clases, lo que detiene el modelo para que funcione mejor.

---

<div style="background-color:#B4DBE9; color:#636363;">
    <h1><center>Conclusion</center></h1>
</div>

1. Los niveles de TDS parecen contener cierta descripción ya que sus valores son en promedio 40 pliegues más que el límite superior para el agua potable segura.

2. Los datos contienen un número casi igual de muestras de agua de nivel de pH ácido y básico.

3. El 92% de los datos se consideraron difíciles.

4. Solo el 2% de las muestras de agua eran seguras en términos de niveles de cloraminas.

5. Solo el 1.8% de las muestras de agua fueron seguras en términos de niveles de sulfato.

6. 90.6% de las muestras de agua tenían niveles de carbono más altos que los niveles típicos de carbono en el agua potable (10 ppm).

7. 76.6% de las muestras de agua fueron seguras para beber en términos de niveles de trihalometano en el agua.

8. 90.4% de las muestras de agua fueron seguras para beber en términos de la turbidez de las muestras de agua.

9. Los coeficientes de correlación entre las características fueron muy bajos.

10. Bosque aleatorio y XGBOost trabajaron lo mejor para entrenar el modelo.

11. El método de conjunto de usar la clase de votación en muestras de plegables K estratificadas dio una precisión de> 64%

---